In [1]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

### Retrieve the data 

In [2]:
#Load all Files (they must be in data directory in a brother directory of the notebook)
data_load = {
    'dipole_moments': pd.read_csv('./data/magnetic_shielding_tensors.csv'),
    'magnetic_shielding_tensors': pd.read_csv('./data/dipole_moments.csv'),
    'mulliken_charges': pd.read_csv('./data/mulliken_charges.csv'),
    'potential_energy': pd.read_csv('./data/potential_energy.csv'),
    'sample_submission': pd.read_csv('./data/sample_submission.csv'),
    'scalar_coupling_contributions': pd.read_csv('./data/scalar_coupling_contributions.csv'),
    'structures': pd.read_csv('./data/structures.csv'),
    'train': pd.read_csv('./data/train.csv'), 
    'test': pd.read_csv('./data/test.csv')
    }

Remark: besides train features, data are not provided in the test dataset.

Should we use those as generative features for the scalar coupling constant?"

## Data preparation

### Objectives
* We try to test the following idea: we will train the model with as an input:
 - the couple of atoms nature and positions
 - the position of all other atoms in the molecule, by decreasing distance from the barycentre of the 2 considered atoms
and as ouputs, the J value but also the other properties that are provided
* Create for each sample, ie each couple of atoms for which a coupling constant is provided, the 1D features for standard RNN and the series of molecules atoms positions for the RNN part of the model
* Prepare the additional output, to improve training

For performance improvement, the idea is to create the atoms position in the molecule during the mini-batch creation, not in one go

In [34]:
train_all = data_load['train']
MAX_MOL_ATOMS_NB = max(train_all.atom_index_0.max(),train_all.atom_index_1.max()) + 1


train_all =  pd.get_dummies(train_all, prefix = 'coupling_type', columns = ['type'])

COUPLING_TYPE_NB = 8



train_all.head(25)
# we have a molecule of methane
# strangely, scalar coupling between identical atoms (we should have a symetric molecule) are slightly different (7 per 100,000)

,id,molecule_name,atom_index_0,atom_index_1,scalar_coupling_constant,coupling_type_1JHC,coupling_type_1JHN,coupling_type_2JHC,coupling_type_2JHH,coupling_type_2JHN,coupling_type_3JHC,coupling_type_3JHH,coupling_type_3JHN
0,0,dsgdb9nsd_000001,1,0,84.80760,1,0,0,0,0,0,0,0
1,1,dsgdb9nsd_000001,1,2,-11.25700,0,0,0,1,0,0,0,0
2,2,dsgdb9nsd_000001,1,3,-11.25480,0,0,0,1,0,0,0,0
3,3,dsgdb9nsd_000001,1,4,-11.25430,0,0,0,1,0,0,0,0
4,4,dsgdb9nsd_000001,2,0,84.80740,1,0,0,0,0,0,0,0
5,5,dsgdb9nsd_000001,2,3,-11.25410,0,0,0,1,0,0,0,0
6,6,dsgdb9nsd_000001,2,4,-11.25480,0,0,0,1,0,0,0,0
7,7,dsgdb9nsd_000001,3,0,84.80930,1,0,0,0,0,0,0,0
8,8,dsgdb9nsd_000001,3,4,-11.25430,0,0,0,1,0,0,0,0
9,9,dsgdb9nsd_000001,4,0,84.80950,1,0,0,0,0,0,0,0


In [4]:
MAX_MOL_ATOMS_NB

29

In [5]:
#Confirmation with the molecule structure
structures = data_load['structures']


# Create one_hot_encoding for the atom type
structures = pd.get_dummies(structures, prefix = 'atom', columns = ['atom'])
structures.head()

,molecule_name,atom_index,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0
1,dsgdb9nsd_000001,1,0.002150,-0.006031,0.001976,0,0,1,0,0
2,dsgdb9nsd_000001,2,1.011731,1.463751,0.000277,0,0,1,0,0
3,dsgdb9nsd_000001,3,-0.540815,1.447527,-0.876644,0,0,1,0,0
4,dsgdb9nsd_000001,4,-0.523814,1.437933,0.906397,0,0,1,0,0


### Function to create a 2D matrix representing all atoms position for a particular molecule
The function should order the atoms in decreasing distance from the A-B atoms considered

#### Question 1: should we keep the atoms of the coupling in the list of atoms of the molecule ?
I think we do, because we want this list of atoms to predict the global variables of molecule, independently of the coupling prediction

#### Question 2: for J1 coupling, in which order should we return the coupled molecules (as we order based on the center, the order will be arbitrary - based on rounding error)


In [6]:
from timeit import default_timer as timer
from lib import data_preparation

# Test for create_couple_env

atom_col = ['x', 'y', 'z', 'atom_C','atom_F','atom_H','atom_N','atom_O']

BATCH_SIZE = 10

train_index = [57, 65, 100, 12, 5000, 168, 758, 9999, 122, 0]
# Warning:the order is not kept by my function: indices will be reordered in ascending order
train_cols = [col for col in train_all.columns if col not in ['id', 'scalar_coupling_constant']]


start = timer()

train_batch = train_all.loc[train_all['id'].isin(train_index), train_cols]
coupled_atoms_feat, atom_pos_feat = data_preparation.create_coupling_batch(structures, train_batch)

end = timer()

print ("Time for", BATCH_SIZE, "samples:", end - start)
train_batch = pd.DataFrame(coupled_atoms_feat, columns = [atom_col+atom_col+['coupling_type_1JHC','coupling_type_1JHN',
                                                          'coupling_type_2JHC','coupling_type_2JHH',
                                                          'coupling_type_2JHN','coupling_type_3JHC',
                                                          'coupling_type_3JHH','coupling_type_3JHN']])
train_batch.head(5)

rnn_train_sample = pd.DataFrame(atom_pos_feat[5, :, :], columns = [atom_col])

rnn_train_sample.head(15)


hello world
Time for 10 samples: 0.4697366790000004


,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O
0,0.740872,-1.620250,-1.275165,0.0,0.0,1.0,0.0,0.0
1,0.220234,-0.190512,-2.177290,0.0,0.0,1.0,0.0,0.0
2,1.758512,-0.173766,-1.308715,0.0,0.0,1.0,0.0,0.0
3,0.721691,-0.525834,-1.262306,1.0,0.0,0.0,0.0,0.0
4,-0.558402,1.948312,-0.838161,0.0,0.0,1.0,0.0,0.0
5,0.979560,1.964591,0.030984,0.0,0.0,1.0,0.0,0.0
6,-1.010706,-0.384570,0.020518,0.0,0.0,1.0,0.0,0.0
7,-0.542523,1.901535,0.930057,0.0,0.0,1.0,0.0,0.0
8,0.515228,-0.368402,0.882311,0.0,0.0,1.0,0.0,0.0
9,0.012153,0.010922,-0.016033,1.0,0.0,0.0,0.0,0.0


## Create the models

In [7]:
tf.keras.backend.clear_session()
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
coupled_atoms_feat.shape

(10, 24)

#### Split train and evaluation sets, at molecule level (it is how the test set is built)

In [38]:
train_cols = [col for col in train_all.columns if col not in ['id','scalar_coupling_constant']]
test_cols = ['scalar_coupling_constant']

validation_ratio = 0.2

molecule_list = train_all['molecule_name'].unique()

np.random.seed(42)
np.random.shuffle(molecule_list)

len(molecule_list)

train_mol, eval_mol = np.split(molecule_list, [int((1-validation_ratio)*len(molecule_list))])

X_train = train_all.loc[train_all['molecule_name'].isin(train_mol), train_cols]
Y_train = train_all.loc[train_all['molecule_name'].isin(train_mol), test_cols]

X_eval = train_all.loc[train_all['molecule_name'].isin(eval_mol), train_cols]
Y_eval = train_all.loc[train_all['molecule_name'].isin(eval_mol), test_cols]

print('Number of samples: \nfor train set:', len(X_train), '\nfor eval set:', len(X_eval))

Number of samples: 
for train set: 3728272 
for eval set: 929875


,scalar_coupling_constant
16,-9.94641
46,87.63260
47,-6.48291
48,-10.60980
49,1.43077


In [35]:
X_eval.shape

(929875, 11)

#### Model for coupling features (simple DNN)

In [9]:
coupling_inputs = keras.Input(shape =(24 ,) )

x = layers.Dense(32, activation='relu')(coupling_inputs)
coupling_int_outputs = layers.Dense(32, activation='relu')(x)

coupling_partial_model = keras.Model(coupling_inputs, coupling_int_outputs, name = 'partial_coupling_model')

coupling_encoding = coupling_partial_model(coupling_inputs)

coupling_output = layers.Dense(1)(coupling_encoding)
coupling_model = keras.Model(coupling_inputs, coupling_output, name =  'coupling_model' )

coupling_partial_model.summary()
coupling_model.summary()

Model: "partial_coupling_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                800       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
Total params: 1,856
Trainable params: 1,856
Non-trainable params: 0
_________________________________________________________________
Model: "coupling_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
partial_coupling_model (Mode (None, 32)                1856      
__________________

#### Training of the coupling model

Submissions are evaluated on the Log of the Mean Absolute Error, calculated for each scalar coupling type, and then averaged across types, so that a 1% decrease in MAE for one type provides the same improvement in score as a 1% decrease for another type.

score=1T∑t=1Tlog(1nt∑i=1nt|yi−yi^|)
Where:

T is the number of scalar coupling types
nt is the number of observations of type t
yi is the actual scalar coupling constant for the observation
yi^ is the predicted scalar coupling constant for the observation
For this metric, the MAE for any group has a floor of 1e-9, so that the minimum (best) possible score for perfect predictions is approximately -20.7232.

In [10]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
coupling_model.compile(loss = 'mean_absolute_error', optimizer=adam, metrix = 'mean_absolute_error')

EPOCHS = 2






#### Features description

* dipole_moments.csv - contains the molecular electric dipole moments. These are three dimensional vectors that indicate the charge distribution in the molecule. The first column (molecule_name) are the names of the molecule, the second to fourth column are the X, Y and Z components respectively of the dipole moment.
* magnetic_shielding_tensors.csv - contains the magnetic shielding tensors for all atoms in the molecules. The first column (molecule_name) contains the molecule name, the second column (atom_index) contains the index of the atom in the molecule, the  third to eleventh columns contain the XX, YX, ZX, XY, YY, ZY, XZ, YZ and ZZ elements of the tensor/matrix respectively.
* mulliken_charges.csv - contains the mulliken charges for all atoms in the molecules. The first column (molecule_name) contains the name of the molecule, the second column (atom_index) contains the index of the atom in the molecule, the third column (mulliken_charge) contains the mulliken charge of the atom.
* potential_energy.csv - contains the potential energy of the molecules. The first column (molecule_name) contains the name of the molecule, the second column (potential_energy) contains the potential energy of the molecule.
* scalar_coupling_contributions.csv - The scalar coupling constants in train.csv (or corresponding files) are a sum of four terms. scalar_coupling_contributions.csv contain all these terms. The first column (molecule_name) are the name of the molecule, the second (atom_index_0) and third column (atom_index_1) are the atom indices of the atom-pair, the fourth column indicates the type of coupling, the fifth column (fc) is the Fermi Contact contribution, the sixth column (sd) is the Spin-dipolar contribution, the seventh column (pso) is the Paramagnetic spin-orbit contribution and the eighth column (dso) is the Diamagnetic spin-orbit contribution.

### Types of Coupling considered

In [ ]:
plt1 = data_load['train'].groupby('type').id.count()

fig1 = plt1.plot.bar(rot = 45)

# up to 3J coupling, only H-C, H-N, H-H coupling

In [ ]:
data_load['test'].groupby('type').id.count()

# This is the same for test :)

In [ ]:
import seaborn as sns

sns.pairplot(data_load['train'][["type", "scalar_coupling_constant", "atom_index_0"]], hue="type", height=5)

# The coupling type seems to be a rather good predictor of the scalar coupling constant, 
# except for 1JHC and 1JHH which have large distributions

# biggest molecule seems to have 29 atoms

### Do we have missing values?

In [ ]:
def create_couple_env_vec(molecule_structures, coupled_atoms, COUPLING_TYPE_NB = 8, MAX_MOL_ATOMS_NB = 29):
    
    ### coupled_atomes has 4 columns: molecule_name, atom_index_0, atom_index_1, type

    batch_size = coupled_atoms.shape[0]
    
    atom_col = ['x', 'y', 'z', 'atom_C','atom_F','atom_H','atom_N','atom_O']
    drop_col = ['molecule_name', 'atom_index'] 
    
    # Select all molecules in the batch first (to quicken the operations)
    molecules_name = coupled_atoms[['molecule_name']].values.reshape(-1)
    molecules = molecule_structures.loc[molecule_structures['molecule_name'].isin(molecules_name)]
    
    # create the output matrices  
    coupled_atoms_feat = np.zeros((batch_size, 8 * 2 + COUPLING_TYPE_NB))
    atom_pos_feat = np.zeros((batch_size, MAX_MOL_ATOMS_NB, len(atom_col)))
    
    # Loop over all samples in the batch
    for i in np.arange(batch_size):
        molecule_name = coupled_atoms.iloc[i,0]
        atom_index_A  = coupled_atoms.iloc[i,1]
        atom_index_B  = coupled_atoms.iloc[i,2]
        coupling_type = coupled_atoms.iloc[i,-COUPLING_TYPE_NB:].values.reshape(1,COUPLING_TYPE_NB )
        
        molecule = molecules.loc[(molecules['molecule_name'] == molecule_name)]
        atom_1 = molecule.loc[(molecule['atom_index'] == atom_index_A)].drop(drop_col, axis = 1).values
        atom_2 = molecule.loc[(molecule['atom_index'] == atom_index_B)].drop(drop_col, axis = 1).values

        coupled_atoms_feat[i,:] = np.concatenate((atom_1, atom_2, coupling_type ), axis = 1)
        
        # calculate the mean position between the 2 coupled_atoms
        p_mean = molecule.loc[(molecule['atom_index'] == atom_index_A) |
                             (molecule['atom_index'] == atom_index_B), ['x', 'y', 'z']].mean(axis = 0)
        
        # retrieve the atom features of all atoms in the molecule
        mol_atom_pos = molecule[atom_col].copy()
           
        mol_atom_pos['distance'] = mol_atom_pos.apply(lambda row : 
                                  np.sqrt((row['x']-p_mean['x'])**2 + (row['y']-p_mean['y'])**2 + (row['z']-p_mean['z'])**2)
                                  , axis = 1)
        mol_atom_pos = mol_atom_pos.sort_values('distance',ascending = False).drop(['distance'], axis = 1)
        atom_pos_feat[i, :mol_atom_pos.shape[0], :mol_atom_pos.shape[1]] = mol_atom_pos.values
    
    return coupled_atoms_feat, atom_pos_feat

In [ ]:
data_load['train'].isna().sum()

# No we don't

#### Molecules structures provided

In [ ]:
print('Total number of molecules provided:', len(structures.groupby('molecule_name')))
print('Total number of molecules in the train set:', len(data_load['train'].groupby('molecule_name')))
print('Total number of molecules in the test set:', len(data_load['test'].groupby('molecule_name')))
# Structures provide all molecules for train and test data sets